In [1]:
import os,cv2,time
import numpy as np

In [2]:
os.sys.path.append("../utilities/")
os.sys.path.append("mobileface_det_yolov3/")


In [3]:
import support_utility_openvino
from yolov3_util import get_objects,filter_objects

In [4]:
output_support = support_utility_openvino.create_plot()

In [5]:
model_path = "/media/prateek/prateek_space/intel_model_dump/public/mobilefacedet-v1-mxnet/openvino_model/mobilefacedet_v1_mxnet-0000.xml"
mask_model_path = "/media/prateek/prateek_space/model_files/mask_detection/mobilenet/basemaskmodel_mobilenet.xml"

In [6]:
model_facedet = support_utility_openvino.async_infer()

Available Devices :  ['CPU', 'GNA', 'GPU']


In [7]:
model_facedet.load_model(model_path,"CPU",input_format="NHWC")

OPTIMIZATION_CAPABILITIES for CPU:  ['FP32', 'FP16', 'INT8', 'BIN']
model inputs : dict_keys(['data'])
model outputs :  dict_keys(['yolov30_yolooutputv30_conv0_fwd/YoloRegion', 'yolov30_yolooutputv31_conv0_fwd/YoloRegion', 'yolov30_yolooutputv32_conv0_fwd/YoloRegion'])


In [8]:
mask_det = support_utility_openvino.async_infer(ie_network=model_facedet.ie)

Available Devices :  ['CPU', 'GNA', 'GPU']


In [9]:
mask_det.load_model(mask_model_path,"GPU")

OPTIMIZATION_CAPABILITIES for GPU:  ['FP32', 'BIN', 'FP16']
model inputs : dict_keys(['input_1_2'])
model outputs :  dict_keys(['act_softmax_2/Softmax'])


In [10]:
layer_params = []
for layer_name in model_facedet.out_blob:
    layer_params.append(model_facedet.net.layers[layer_name].params)

In [23]:
channel = "/media/prateek/prateek_space/dataset/jayshree rishra textlie/Worsted Cam-6 (13.07.20).mp4"
cap  = cv2.VideoCapture(channel)

thresh = .3
iou_thresh = .4
color = (255,0,0)
frame_height, frame_width = (int(cap.get(4)),int(cap.get(3)))
cv2.namedWindow("preview",cv2.WINDOW_NORMAL)
model_facedet.reinit_model()
# mask_det.reinit_model()
mask_labels = ["mask","no_mask"]
while True:
    t1 = time.time()
    ret,frame = cap.read()
    if not ret:
        break
    op = model_facedet.predict(frame)
    if model_facedet.frame_processed>=model_facedet.num_requests:
        frame,attr,res = model_facedet.postprocess_op()
        objects = get_objects(res,layer_params,(model_facedet.height,model_facedet.width),(frame_height, frame_width),thresh,False)
        objects = filter_objects(objects,iou_thresh,thresh)
        if len(objects)>0:

            res = [[1,1,loc["confidence"],loc["xmin"]/frame_width,loc["ymin"]/frame_height,loc["xmax"]/frame_width,loc["ymax"]/frame_height]  for loc in objects]
            res = np.reshape(res,(1,1,len(res),7))
            op,updated_box,res_filt = output_support.trim_frame_with_result(frame,res,offset=5,return_results=True,threshold=thresh)
            res_masks = mask_det.predict_batch_async(op)
            mask_pred_labels = [res_mask[0].argmax() for res_mask in res_masks]
            for bbox,label in zip(updated_box,mask_pred_labels):
                color = (0,255,0)
                if label==1:
                    color = (0,0,255)
                frame = output_support.create_bbox(frame,[tuple(bbox[:2]),tuple(bbox[2:])],color)
            frame = output_support.write_text(frame,"FPS :%.2f"%(1/(time.time()-t1)),text_color=(0, 0, 0),font_scale=1)

#         frame = output_support.label_obj_detection(frame,res,labels=["face"],thresh=thresh)
        
#         for loc in objects:
#             xmin  = max(loc["xmin"],0)
#             ymin  = max(loc["ymin"],0)
#             xmax = min(loc["xmax"],frame_width)
#             ymax = min(loc["ymax"],frame_height)
# #             output_support.trim_frame_with_bboxes
#             cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), color, 2)
        cv2.imshow("preview",frame)
        k = cv2.waitKey(1)
        if k==ord('q'):
            break
            
cv2.destroyAllWindows()
cap.release()
            

            
            

In [13]:
for layer_name in model_facedet.out_blob:
    print(model_facedet.net.layers[model_facedet.net.layers[layer_name].parents[0]].out_data[0].shape)

[1, 18, 8, 8]
[1, 18, 16, 16]
[1, 18, 32, 32]


In [9]:
channel = "/home/prateek/prateek_space/dataset/veraval_production/videos/face_veraval/202003211445.mp4"
cap  = cv2.VideoCapture(channel)
ret,frame = cap.read()
objects = get_objects(op,layer_params,(256,256),frame.shape[:-1],.4,False)

KeyError: 0

In [8]:
op = model_facedet.predict_sync(frame)

In [7]:
channel = "/home/prateek/prateek_space/dataset/veraval_production/videos/face_veraval/202003211445.mp4"
cap  = cv2.VideoCapture(channel)
cv2.namedWindow("preview",cv2.WINDOW_NORMAL)
while True:
    ret,frame = cap.read()
    break
    if not ret:
        break
    detection, detect_masks = detector.detect(frame)
    for i, (score, xmin, ymin, xmax, ymax) in enumerate(zip(*detection.face_detection)):
        xmin = int(max(0, xmin))
        ymin = int(max(0, ymin))
        xmax = int(min(frame.shape[1], xmax))
        ymax = int(min(frame.shape[0], ymax))
        color = (255, 0, 0)
        cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), color, 2)
        cv2.putText(frame, str(round(score * 100, 1)) + ' %', (xmin, ymin - 7),
                     cv2.FONT_HERSHEY_COMPLEX, 0.6, color, 1)
        for j in range(5):
            x = detection.landmarks_regression[0][i, j].astype(np.int)
            y = detection.landmarks_regression[1][i, j].astype(np.int)
            cv2.circle(frame, (x, y), 2, (0, 255, 255), 2)

    cv2.putText(frame, 'summary: {:.1f} FPS'.format(
        1.0 / detector.infer_time), (5, 15), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 200))
    cv2.imshow("preview",frame)
    k = cv2.waitKey(1)
    if k==ord('q'):
        break


cv2.destroyAllWindows()
cap.release()

In [ ]:
model

In [14]:
model_facedet.exec_net.input_info['data'].input_data.shape[1:3]

[256, 256]